In [1]:
# coding: utf-8
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pylab as plt
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, Matern, ConstantKernel as C

from scipy.stats import multivariate_normal
from tqdm import tqdm_notebook as tqdm

import pandas as pd
%matplotlib inline

import os
import copy
import json
import subprocess
import pickle

def mkdir_if_not_exist(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    else:
        pass

In [2]:
class GPUCB(object):

  def __init__(self, meshgrid, environment, beta=100., noise=False):
    '''
    meshgrid: Output from np.methgrid.
    e.g. np.meshgrid(np.arange(-1, 1, 0.1), np.arange(-1, 1, 0.1)) for 2D space
    with |x_i| < 1 constraint.
    environment: Environment class which is equipped with sample() method to
    return observed value.
    beta (optional): Hyper-parameter to tune the exploration-exploitation
    balance. If beta is large, it emphasizes the variance of the unexplored
    solution solution (i.e. larger curiosity)
    '''
    self.meshgrid = np.array(meshgrid)
    self.environment = environment
    self.beta = beta

    self.X_grid = self.meshgrid.reshape(self.meshgrid.shape[0], -1).T
    self.mu = np.array([0. for _ in range(self.X_grid.shape[0])])
    self.sigma = np.array([0.5 for _ in range(self.X_grid.shape[0])])

    if self.X_grid.shape[1] == 2:
        nrow, ncol = self.meshgrid.shape[1:]
        self.z = self.environment.sample(self.X_grid.T).reshape(nrow, ncol) 
    
    self.X = []
    self.T = []
    self.kernel_list = []
    
    # Instanciate a Gaussian Process model
    my_kernel  = C(1, constant_value_bounds="fixed")* RBF(2, length_scale_bounds="fixed") # works well, but not so sharp
    #     my_kernel = Matern(nu=2.5) # good
    if noise:
        my_kernel += WhiteKernel(1e-1)

    self.gp = GaussianProcessRegressor(kernel=my_kernel)
    
  def argmax_ucb(self):
    ucb = np.argmax(self.mu + self.sigma * np.sqrt(self.beta))
    return ucb

  def learn(self):
    grid_idx = self.argmax_ucb()
    self.sample(self.X_grid[grid_idx])
    
    self.gp.fit(self.X, self.T)
    self.kernel_list.append(self.gp.kernel_)
    self.mu, self.sigma = self.gp.predict(self.X_grid, return_std=True)

  def sample(self, x):
    t = self.environment.sample(x)
    self.X.append(x)
    self.T.append(t)

  def plot(self, output_dir):
    
    if self.X_grid.shape[1] != 2:
        print ("plotting only supports X_grid that consisted of 2 dimentions.")
        return 
    
    fig = plt.figure()
    ax = Axes3D(fig)
    
    ax.plot_wireframe(self.meshgrid[0], self.meshgrid[1],
        self.mu.reshape(self.meshgrid[0].shape), alpha=0.5, color='g')
    
    ucb_mesh = self.mu + self.sigma * np.sqrt(self.beta)
    ax.plot_wireframe(self.meshgrid[0], self.meshgrid[1],
        ucb_mesh.reshape(self.meshgrid[0].shape), alpha=0.5, color='y')
    
#     ax.plot_wireframe(self.meshgrid[0], self.meshgrid[1], self.z, alpha=0.3, color='b')
    
    
    ax.scatter([x[0] for x in self.X], [x[1] for x in self.X], self.T, c='r',
        marker='o', alpha=1.0)
    
    out_fn = os.path.join(output_dir, 'res_%04d.png' % len(self.X))
    mkdir_if_not_exist(output_dir)
    
    plt.savefig(out_fn)
    plt.close()

In [3]:
from abc import ABCMeta, abstractmethod

class BasicEnvironment(object):
    
    __metaclass__ = ABCMeta
    
    def __init__(self, sorted_keys, result_filename='result.csv', output_dir='output'):
        self.result_filename = result_filename     
        
        if os.path.exists(result_filename) :
            print("Oops! %s has already existed... Please change the filename or set reload flag to be true!"%result_filename)
            raise AttributeError

        
        self.hyper_param_names = sorted_keys
        
        with open(result_filename, 'w') as f:
            columns = self.hyper_param_names + ['output']
            f.write(','.join(columns) + os.linesep)
        
        self.df = pd.read_csv(result_filename)
        
        mkdir_if_not_exist(output_dir)
        self.output_dir = output_dir
        
    
    @abstractmethod 
    def run_model(self, *args, **kargs):
        pass
    
    def preprocess_param(self, x):
        return np.array(x)
    
    def sample(self, x):        
        self.df = pd.read_csv(self.result_filename)
        n_model = self.df.shape[0] + 1

        x = self.preprocess_param(x)
        
        prefix_msg = 'No.%04d model started!  '%n_model
        pair_msg = ', '.join(['{}: {}'.format(k, v) for k,v in zip(self.hyper_param_names, x)])
        print (prefix_msg + pair_msg)
        
        
        result = self.run_model(n_model, *x)
        
        self.df.loc[len(self.df)] = list(x) + [result]  
        self.df.to_csv(self.result_filename, index=False)
        
        msg = 'No.%04d model finished! Result was %f'%(n_model, result)
        print (msg)
        
        return result



class LDA_Environment(BasicEnvironment):
    
    def __init__(self, sorted_keys, lda_output_filename, output_dir, original_config, default_n_cluster=8, lda_n_iter=400):
        super().__init__(sorted_keys, lda_output_filename, output_dir)
        
        self.original_config = original_config
        self.default_n_cluster = default_n_cluster
        self.lda_n_iter = lda_n_iter
    
    def preprocess_param(self, x):
        # key order must be alpha, beta, n_cluster
        y = np.ones_like(x)
        y[:2] = 10 ** x[:2]
        y[2:] = x[2:]
        return y
    
    def set_my_config(self, model_number, alpha, beta, n_cluster):
        config = copy.deepcopy(self.original_config)

        config['filename_result'] = './output/output%04d.txt'%model_number
        config['filename_model'] = './output/model%04d'%model_number
        config["pathname_dump"] =  "./dump/dump%04d/"%model_number

        config['ALPHA'] = alpha
        config['TYPE_LIST'][0] = "Categorical:0,1,2,3,4,5,6,7,8,9;%f"%beta
        config['CLUSTER_NUM'] = n_cluster
        config['end'] = self.lda_n_iter
        
        mkdir_if_not_exist(config["pathname_dump"])

        return config
    
    def run_lda(self, model_number, alpha, beta, n_cluster):
        
        conf = self.set_my_config(model_number, alpha, beta, n_cluster)
        conf_fn = os.path.join(self.output_dir, 'conf%04d.json'%model_number)
        
        with open(conf_fn, "w") as f:
            json.dump(conf, f, ensure_ascii=False, indent=4, sort_keys=True, separators=(',', ': '))
        
        cmd = "java -jar IndependentMixtureModelMCMC.jar training %s"%conf_fn

        subprocess.call(cmd, shell=True)

        loglikelihood = np.loadtxt(conf['pathname_dump'] + 'loglikelihood.dmp')[-1]
        
        return loglikelihood

    def run_model(self, model_number, alpha, beta, n_cluster):
        res = self.run_lda(model_number, alpha, beta, n_cluster)
        return res

In [4]:
original_config_dict = {
 "filename_training": "./input/input.txt",
 "filename_result": "./output/output.txt",
 "filename_testing": "./input/input.txt",
 "filename_model": "./output/model",
 "pathname_dump": "./dump/",
 "HAS_ID": 0,
 "ALPHA": 1,
 "FEATURE_NUM": 1,
 "CLUSTER_NUM": 8,
 "DATA_SIZE": 0,
 "DOC_SIZE": 0,
 "THREAD_NUM": 2,
 "start": 0,
 "end": 1000,
 "TYPE_LIST": [
  "Categorical:0,1,2,3,4,5,6,7,8,9;1"
 ]
}

my_hyper_param_dic = {
    "alpha": np.arange(-2, 2.01, 0.2),
    "beta": np.arange(-2, 2.01, 0.2),
    "n_cluster": np.arange(5, 20.1).astype(int)
}

my_sorted_keys = sorted(my_hyper_param_dic.keys())

print (my_sorted_keys)

output_dir = 'output'
# reload_model_filename = os.path.join(output_dir, 'agent.pkl')
# reload_model_filename = None
reload_model_filename = os.path.join(output_dir, 'learned_agent.pkl')

result_filename = 'lda_result.csv'

if reload_model_filename is not None:
    with open(reload_model_filename, mode='rb') as f:
        agent = pickle.load(f)

else:
    if os.path.exists(result_filename):
        os.remove(result_filename)
    env = LDA_Environment(my_sorted_keys, result_filename, output_dir, original_config_dict, lda_n_iter=400)
    agent = GPUCB(np.meshgrid(*[my_hyper_param_dic[k] for k in my_sorted_keys]), env, 36)

output_model_filename = os.path.join(output_dir, 'learned_agent.pkl')


n_iter = 100

for i in tqdm(range(n_iter)):
    try:
        agent.learn()
    except KeyboardInterrupt:
        print ("Learnig process was forced to stop!")
        with open(output_model_filename, mode='wb') as f:
            pickle.dump(agent, f)
            print ("%s was saved."%output_model_filename)
        
        break


In [8]:
# TODO: java.lang.ArrayIndexOutOfBoundsException